<a href="https://colab.research.google.com/github/jaegertros/cov-d-19-data-notebooks/blob/main/scripts/notebooks/jh_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Johns Hopkins data

## Dependencies

In [ ]:
import pandas as pd
import os
from datetime import datetime

INPUT_PATH = 'input/jh/'
OUTPUT_PATH = 'public/data/jh/'

POPULATION_CSV_PATH = os.path.join(INPUT_PATH, 'population_thousands.csv')
REGIONS_CSV_PATH = os.path.join(INPUT_PATH, 'regions.csv')
CASES_CSV_PATH = os.path.join(INPUT_PATH, 'cases.csv')
DEATHS_CSV_PATH = os.path.join(INPUT_PATH, 'deaths.csv')
RECOVERED_CSV_PATH = os.path.join(INPUT_PATH, 'recovered.csv')

INDEX_COLUMNS = ['Country/Region', 'Province/State']

In [ ]:
!mkdir -p $OUTPUT_PATH

## Download

Download time series data from [GitHub repository](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series), which is updated daily.

In [ ]:
!curl -Lo $INPUT_PATH/cases.csv https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv
!curl -Lo $INPUT_PATH/deaths.csv https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv
!curl -Lo $INPUT_PATH/recovered.csv https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   210  100   210    0     0    995      0 --:--:-- --:--:-- --:--:--   995
100 74283  100 74283    0     0   193k      0 --:--:-- --:--:-- --:--:--  193k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   207  100   207    0     0   1182      0 --:--:-- --:--:-- --:--:--  1182
100 68994  100 68994    0     0   196k      0 --:--:-- --:--:-- --:--:-- 2260k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   210  100   210    0     0   1329      0 --:--:-- --:--:-- --:--:--  1329
100 71320  100 71320    0     0   212k      0 --:--:-- --:--:-- --:--:--  212k


## Standardize

Load the regions with OWID name mappings

In [ ]:
df_regions = pd.read_csv(REGIONS_CSV_PATH, index_col=INDEX_COLUMNS)

Load Country/Region/States from Johns Hopkins data and check that we have accounted for all

In [ ]:
df_current_regions = pd.concat([
    pd.read_csv(path)[INDEX_COLUMNS]
    for path in [
        CASES_CSV_PATH,
        DEATHS_CSV_PATH,
        RECOVERIES_CSV_PATH
    ]
]) \
.drop_duplicates() \
.sort_values(by=INDEX_COLUMNS) \
.merge(
    df_regions,
    how='left',
    on=INDEX_COLUMNS
)

In [ ]:
if df_current_regions['OWID Country Name'].isnull().any():
    print("Please add country mappings for: ")
    print(df_current_regions[df_current_regions['OWID Country Name'].isnull()])
    df_current_regions.to_csv('to_standardize.csv', index=False)
    assert False

Load Johns Hopkins data

In [ ]:
def melt(df, var_name):
    return df.melt(
        id_vars=df.columns[0], 
        value_vars=df.columns[1:], 
        var_name='date', 
        value_name=var_name
    ).dropna()

In [ ]:
def us_to_iso_date(datestr):
    return datetime.strptime(datestr, '%m/%d/%y').strftime('%Y-%m-%d')

In [ ]:
def transform_and_standardize(df, var_name):
    df = df.drop(columns=['Lat', 'Long'])
    df = df.merge(
        df_regions,
        how='left',
        on=['Country/Region', 'Province/State']
    ).rename(columns={'OWID Country Name': 'location'})
    if df['location'].isnull().any():
        print("missing mappings in %s for:" % var_name)
        print(df[df['location'].isnull()][['Country/Region', 'Province/State', 'location']])
        assert False
    df = df.groupby('location').sum().reset_index()
    df = melt_csv(df, var_name)
    df['date'] = df['date'].map(us_to_iso_date)
    return df.sort_values(by=['location', 'date'])

In [ ]:
df_cases = transform_and_standardize(pd.read_csv(CASES_CSV_PATH), 'total_cases')
df_deaths = transform_and_standardize(pd.read_csv(DEATHS_CSV_PATH), 'total_deaths')
df_recovered = transform_and_standardize(pd.read_csv(RECOVERED_CSV_PATH), 'total_recovered')

In [ ]:
df_merged = df_cases \
    .merge(
        df_deaths,
        how='outer',
        on=['date', 'location']
    ) \
    .merge (
        df_recovered,
        how='outer',
        on=['date', 'location']
    )

In [ ]:
df_global = df_merged.groupby('date').sum().reset_index()
df_global['location'] = 'World'

In [ ]:
df_merged = pd.concat([df_merged, df_global], sort=True, ignore_index=True)

In [ ]:
for col in ['cases', 'deaths', 'recovered']:
    df_merged['new_%s' % col] = df_merged.groupby('location')['total_%s' % col].diff().astype('Int64')

In [ ]:
df_merged

,date,location,total_cases,total_deaths,total_recovered,new_cases,new_deaths,new_recovered
0,2020-01-22,Afghanistan,0,0,0,<NA>,<NA>,<NA>
1,2020-01-23,Afghanistan,0,0,0,0,0,0
2,2020-01-24,Afghanistan,0,0,0,0,0,0
3,2020-01-25,Afghanistan,0,0,0,0,0,0
4,2020-01-26,Afghanistan,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
9685,2020-03-14,World,156094,5819,72624,10901,415,2373
9686,2020-03-15,World,167446,6440,76034,11352,621,3410
9687,2020-03-16,World,181527,7126,78088,14081,686,2054
9688,2020-03-17,World,197142,7905,80840,15615,779,2752


In [ ]:
df_grapher = df_merged.copy()
df_grapher['date'] = pd.to_datetime(df_grapher['date']).map(lambda date: (date - datetime(2020, 1, 21)).days)
df_grapher = df_grapher[[
    'location', 'date', 
    'new_cases', 'new_deaths', 'new_recovered',
    'total_cases', 'total_deaths', 'total_recovered']] \
    .rename(columns={
        'location': 'country',
        'date': 'year',
        'new_cases': 'Daily new confirmed cases of COVID-19 (Johns Hopkins)',
        'new_deaths': 'Daily new confirmed deaths due to COVID-19 (Johns Hopkins)',
        'new_recovered': 'Daily new confirmed recoveries from COVID-19 (Johns Hopkins)',
        'total_cases': 'Total confirmed cases of COVID-19 (Johns Hopkins)', 
        'total_deaths': 'Total confirmed deaths due to COVID-19 (Johns Hopkins)',
        'total_recovered': 'Total confirmed recoveries from COVID-19 (Johns Hopkins)',
    })

In [ ]:
df_grapher.to_csv(os.path.join(OUTPUT_PATH, 'grapher.csv'), index=False)